In [1]:
import torch
import sys
sys.path.append('..')
from transformers import AutoModelForCausalLM, AutoTokenizer
import lre.models as models
import lre.functional as functional
import os

import json
import random
from lre.data import Relation, RelationSample, Sequence
import lre.metrics as metrics
import lre.functional as functional

device = 'cuda:0'
weights = []
biases = []
subjects = []
print("loading model")
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-hf",\
                                             torch_dtype=torch.float16, low_cpu_mem_usage=True)
tokenizer.pad_token = tokenizer.eos_token
model.to(device)
#tokenizer.pad_token = tokenizer.eos_token
mt = models.ModelAndTokenizer(model,tokenizer)

/home/exia/miniconda3/envs/my-lre/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


loading model


Loading checkpoint shards: 100%|█████████████████████████████████| 2/2 [00:00<00:00,  6.15it/s]


True

In [11]:
os.chdir('data')
os.listdir()

['W2VBenchmark.py',
 'noc_approx',
 'TestLREJacobian_Llama.py',
 '3_27_approx',
 '5_27_approx_W2V',
 'LLRA.py',
 '6_27_approx_W2V',
 'results',
 'llama_15_31_approx',
 'llama_10_31_approx',
 '9_27_approx',
 '7_27_approx_W2V',
 'gemma',
 'llama_9_31_approx',
 'llama_12_31_approx',
 'llama_5_31_approx',
 'space_viz',
 'llama_13_31_approx',
 'approx',
 'LoadGemma.py',
 'polar_space_viz',
 '9_27_approx_W2V',
 'json',
 'llama_4_31_approx',
 '7_27_approx',
 'RelBenchmark.py',
 'TestOrientedLRE.py',
 '6_27_approx',
 'text',
 '4_27_approx',
 'Make_Weights_GPT-J.py',
 '3_27_approx_W2V',
 'spaced_6_27_approx',
 'llama_16_31_approx',
 'ipynb_experiments',
 'env',
 'llama_6_31_approx',
 'TestLREJacobian.py',
 'llama_11_31_approx',
 'Make_Weights_Llama.py',
 'llama_14_31_approx',
 'llama_8_31_approx',
 '5_27_approx',
 '8_27_approx_W2V',
 'json_paths.txt',
 'spaced_7_27_approx',
 'json2',
 '.ipynb_checkpoints',
 'llama_7_31_approx',
 '8_27_approx',
 '4_27_approx_W2V',
 'ln_approx']

In [14]:
import llra.build as build
from importlib import reload
reload(build)
reload(functional)
build.determine_device(mt)
start, end = 5, 31
APPROX_FOLDER = f'llama_{start}_{end}_approx'
WEIGHT_NAME = f's_o_weight_{start}_{end}'
BIAS_NAME = f's_o_bias_{start}_{end}'

In [17]:
relations = os.listdir(APPROX_FOLDER)

In [16]:
json_path = 'json/enckno/E06 [animal - youth].json'
file = open(json_path, 'r')
data = json.load(file)
file.close()

relation = Relation.from_dict(data)
relation.samples[0].object[0]

'baby'

In [18]:
import llra.build as build
#OBTAIN LRE WEIGHTS: Method 1
#Load weights saved with new naming convention

def find_exact_match(filename, search_string):
    with open(filename, 'r') as file:
        for line in file:
            if search_string.lower() in line.strip().lower():
                return line.strip()
    print(f"Could not find {search_string}")
    return None
    
from baukit.baukit import parameter_names, get_parameter
import torch.nn as nn
from collections import defaultdict

N_ICL = 8 
N_AVG = 8
N_TRIALS = 8
VIEW_SAMPLES = 40

import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

obj_dict = defaultdict(list)

for _ in range(0,8):
    for beta in [1]:
        for relation in relations:
            wdir = f"{APPROX_FOLDER}/{relation}"
            ignored_names = ['.ipynb_checkpoints', 'disappoint']
            j_samples = [x for x in os.listdir(wdir) if x not in ignored_names]
            j_samples = random.sample(j_samples, N_AVG)
            #print(f'using {j_samples} to build LRE')
            # reg_weight = build.mean_weight_or_bias(wdir,
            #                                        WEIGHT_NAME, j_samples).half().to(device)
            # reg_bias = build.mean_weight_or_bias(wdir,
            #                                      BIAS_NAME, j_samples).half().to(device)
            json_path = find_exact_match("json_paths.txt", relation)
            file = open(json_path, 'r')
            data = json.load(file)
            file.close()
            
            relation = Relation.from_dict(data)
            prompt_template = relation.prompt_templates[0]
            
            #ASSEMBLE PROMPTS AND OBJECT ANSWERS
            clozed_prompts = []
            clozed_answers = []
            #Do not use the training examples for ICL or testing purposes.
            test_samples = [x for x in relation.samples if x not in j_samples]
            random.shuffle(test_samples)
            for x in test_samples:
                test_samples_no_x = [t for t in test_samples if t != x]
                samples = [x] + random.sample(test_samples_no_x, N_ICL - 1)
                cloze_prompt = functional.make_prompt(
                    template = prompt_template, 
                    target = x,
                    examples = samples
                    )
                clozed_prompts.append(cloze_prompt)
                clozed_answers.append(x.object)
            
            outputs_lm = functional.predict_next_token(mt=mt, prompt=clozed_prompts)
            preds_lm =  [[x.token for x in xs] for xs in outputs_lm]
            recall_lm = metrics.recall(preds_lm, clozed_answers)
            reg_correct = 0
            no_bias_correct = 0
            first_token_correct = 0
            lm_correct = 0
            for i, sample, objs, prompt, preds in \
            zip(range(0,50), test_samples, clozed_answers, clozed_prompts, preds_lm):
                if (metrics.any_is_nontrivial_prefix(predictions=preds, targets=objs)):
                    # # print(prompt)
                    #uses the regular LRE
                    # reg_hs = build.get_hidden_state(mt, prompt, sample.subject, start)
                    # reg_hs_end = build.get_hidden_state(mt, prompt, sample.subject, end)
                    # reg_hsweight = reg_hs.mm(reg_weight.t())
                    # reg_object_hs = reg_hsweight * beta + reg_bias
                    # reg_preds = build.get_object(mt, reg_object_hs)[0]
                    
                    # no_bias_hs = reg_hsweight
                    # no_bias_preds = build.get_object(mt, no_bias_hs)
    
                    # no_weight_hs = beta * reg_hs + reg_bias
                    # no_weight_preds = build.get_object(mt, no_weight_hs)[0]
                    # init_subj_id = mt.tokenizer(sample.subject).input_ids[0]
                    # init_subj_token = mt.tokenizer.convert_ids_to_tokens(init_subj_id)
                    #print(init_subj_token)
                    
                    # if(metrics.any_is_nontrivial_prefix(predictions=[reg_preds[0]], targets=objs)):
                    #     reg_correct += 1
        
                    # if(metrics.any_is_nontrivial_prefix(predictions=[no_bias_preds[0]], targets=objs)):
                    #     no_bias_correct += 1
                
                    # if(metrics.any_is_nontrivial_prefix(predictions=[no_weight_preds[0]], targets=objs)):
                    #     first_token_correct += 1
                        
                    lm_correct += 1
                    # obj_dict[relation.name].append(sample.object[0])
            print(f'{relation.name},{lm_correct}')
            #print(f'beta,{beta},{relation.name},reg_correct,{reg_correct},no_bias_correct,{no_bias_correct}')

name - nationality,49
animal - youth,36
verb_Ving - 3pSg,49
noun+less_reg,49
verb+able_reg,50
UK_city - county,34
antonyms - binary,47
verb_inf - 3pSg,50
re+verb_reg,50
verb_inf - Ved,50
country - language,48
meronyms - part,47
verb_Ving - Ved,50
animal - shelter,47
hypernyms - misc,46
meronyms - substance,49
noun - plural_irreg,49
un+adj_reg,50
verb+ment_irreg,50
adj+ness_reg,50
over+adj_reg,50
verb+er_irreg,50
adj+ly_reg,50
name - occupation,50
synonyms - intensity,43
animal - sound,36
noun - plural_reg,50
Ving - verb_inf,50
male - female,45
verb_3pSg - Ved,50
meronyms - member,41
things - color,50
hyponyms - misc,44
adj - superlative,47
verb+tion_irreg,50
synonyms - exact,46
hypernyms - animals,50
country - capital,49
name - nationality,46
animal - youth,35
verb_Ving - 3pSg,49
noun+less_reg,49
verb+able_reg,50
UK_city - county,32
antonyms - binary,47
verb_inf - 3pSg,50
re+verb_reg,50
verb_inf - Ved,50
country - language,49
meronyms - part,48
verb_Ving - Ved,50
animal - shelter,45
hy

In [ ]:
obj_dict

In [47]:
for name in obj_dict.keys():
    words = obj_dict[name]
    tokens = [mt.tokenizer(x).input_ids for x in words]
    wdict = {}
    for x in tokens:
        tok1 = mt.tokenizer.decode(x[0])
        if tok1 in wdict.keys():
            wdict[tok1] += 1
        else:
            wdict[tok1] = 1
    print(f'{name},{len(wdict)}')

name - nationality,13
animal - youth,14
verb_Ving - 3pSg,48
noun+less_reg,46
verb+able_reg,47
UK_city - county,16
antonyms - binary,43
verb_inf - 3pSg,46
re+verb_reg,15
verb_inf - Ved,48
country - language,21
meronyms - part,40
verb_Ving - Ved,49
animal - shelter,18
hypernyms - misc,29
meronyms - substance,33
noun - plural_irreg,47
un+adj_reg,7
verb+ment_irreg,41
adj+ness_reg,48
over+adj_reg,4
verb+er_irreg,47
adj+ly_reg,49
name - occupation,20
synonyms - intensity,35
animal - sound,21
noun - plural_reg,48
Ving - verb_inf,49
male - female,39
verb_3pSg - Ved,47
meronyms - member,33
things - color,7
hyponyms - misc,42
adj - superlative,43
verb+tion_irreg,48
synonyms - exact,39
hypernyms - animals,9
country - capital,43


In [27]:
mt.tokenizer.decode(445)

'red'